# ETL_EDA    
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. The data are store (almost) in their raw form.

## 1. Raw Data from LAPD     
The [dataset](https://data.lacity.org/A-Safe-City/Arrest-Data-from-2010-to-Present/yru6-6re4) is a continuously updated file in csv format. It contains City of Los Angeles arrest data from 2010 to present. It can be retrieved simply by making requests without parameters. For more complicated API call, you may need to add query parameters.

In [9]:
import logging
import pandas as pd
from sodapy import Socrata
import utils

# download data
CRIME_SOURCE = "data.lacity.org"
client0 = Socrata(CRIME_SOURCE, None)
results = client0.get("yru6-6re4",where="arst_date > '2019-6-01T00:00:00.000'",limit = 1000000) # wait to be confirmed
print(results[:2])
print('...')
print(results[-2:])

[{'rpt_id': '5646788', 'arst_date': '2019-06-02T00:00:00.000', 'time': '0045', 'area': '06', 'area_desc': 'Hollywood', 'rd': '0646', 'age': '56', 'sex_cd': 'M', 'descent_cd': 'H', 'chrg_grp_cd': '24', 'grp_description': 'Miscellaneous Other Violations', 'arst_typ_cd': 'F', 'charge': '3455(B)1PC', 'chrg_desc': 'WARRANT-PETITION TO REVOKE COMMUNITY SUPVN', 'location': 'SUNSET', 'crsst': 'CHEROKEE', 'location_1': {'latitude': '34.098', 'longitude': '-118.3353', 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}}, {'rpt_id': '5647286', 'arst_date': '2019-06-02T00:00:00.000', 'time': '2010', 'area': '01', 'area_desc': 'Central', 'rd': '0159', 'age': '22', 'sex_cd': 'M', 'descent_cd': 'H', 'chrg_grp_cd': '07', 'grp_description': 'Vehicle Theft', 'arst_typ_cd': 'F', 'charge': '10851(A)VC', 'chrg_desc': "TAKE VEHICLE W/O OWNER'S CONSENT", 'location': 'MILL', 'crsst': 'INDUSTRIAL', 'location_1': {'latitude': '34.0359', 'longitude': '-118.2344', 'human_address': '{"address":

## 2. Raw Data to Documents/Dicts      
Datetime conversion is made and age column is changed to numeric data type. Now the data can be easily converted to a list of dicts which is what we want for the MongoDB.

In [10]:
from datetime import datetime

df = pd.DataFrame.from_records(results)
df['arst_date'] = pd.to_datetime(df['arst_date'])
# df['hour'] = df['time'].apply(lambda x: datetime.strptime(x, '%H%M').time().hour)
#df['month'] = df['arst_date'].apply(lambda x: datetime.strptime(x, '%Y%m').time())
df['age'] = pd.to_numeric(df['age'])
df.head()

,rpt_id,arst_date,time,area,area_desc,rd,age,sex_cd,descent_cd,chrg_grp_cd,grp_description,arst_typ_cd,charge,chrg_desc,location,crsst,location_1
0,5646788,2019-06-02,0045,06,Hollywood,0646,56,M,H,24,Miscellaneous Other Violations,F,3455(B)1PC,WARRANT-PETITION TO REVOKE COMMUNITY SUPVN,SUNSET,CHEROKEE,"{'latitude': '34.098', 'longitude': '-118.3353..."
1,5647286,2019-06-02,2010,01,Central,0159,22,M,H,07,Vehicle Theft,F,10851(A)VC,TAKE VEHICLE W/O OWNER'S CONSENT,MILL,INDUSTRIAL,"{'latitude': '34.0359', 'longitude': '-118.234..."
2,5646925,2019-06-02,0130,10,West Valley,1023,31,M,H,12,Weapon (carry/poss),F,25850(A)PC,CARRY LOADED F/ARM PRSON/HEH PUBLIC PLACE,19000 VANOWEN ST,NaN,"{'latitude': '34.1939', 'longitude': '-118.547..."
3,5647265,2019-06-02,1350,01,Central,0143,30,M,B,06,Larceny,M,459.5PC,SHOPLIFTING,400 S BROADWAY,NaN,"{'latitude': '34.0495', 'longitude': '-118.249..."
4,5646978,2019-06-02,0930,13,Newton,1351,28,F,B,04,Aggravated Assault,F,245(A)(1)PC,"ADW, NOT FIREARM, W/GBI",100 W 43RD PL,NaN,"{'latitude': '34.0047', 'longitude': '-118.274..."


## 3. Upsert MongoDB         
To avoid duplicate data entries between each run, the de-duplication mechanism makes sure that only a new record with a different report_id (rpt_id) will be inserted to the database. 

In [11]:
import pymongo
client = pymongo.MongoClient()

In [12]:
db = client.get_database("crime")
collection = db.get_collection("crime")
update_count = 0
for record in df.to_dict('records'):
    result = collection.replace_one(
        filter = {'rpt_id': record['rpt_id']},    # locate the document if exists
        replacement = record,                         # latest document
        upsert=True)
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df.shape[0]}, update={update_count}, "
      f"insert={df.shape[0]-update_count}")

KeyboardInterrupt: 